In [1]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
# ensure the below library is installed on your fabric environment
import reverse_geocoder as rg

StatementMeta(, fcdfbde8-9474-44d7-a658-c09804d4757e, 5, Finished, Available, Finished)

In [7]:
#sample = spark.read.table("earthquake_events_silver").take(1)
#lat = sample[0]['latitude']
#lon = sample[0]['longitude']
#coordinates = (float(lat), float(lon))
#rg.search(coordinates)

StatementMeta(, fcdfbde8-9474-44d7-a658-c09804d4757e, 11, Finished, Available, Finished)

Loading formatted geocoded file...


[{'lat': '38.82213',
  'lon': '-122.72305',
  'name': 'Cobb',
  'admin1': 'California',
  'admin2': 'Lake County',
  'cc': 'US'}]

In [8]:
df = spark.read.table("earthquake_events_silver").filter(col('time') > start_date)

StatementMeta(, fcdfbde8-9474-44d7-a658-c09804d4757e, 12, Finished, Available, Finished)

In [9]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    coordinates = (float(lat), float(lon))
    return rg.search(coordinates)[0].get('cc')

    # registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

# adding country_code and city attributes
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

StatementMeta(, fcdfbde8-9474-44d7-a658-c09804d4757e, 13, Finished, Available, Finished)

In [14]:
# adding significance classification
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

StatementMeta(, fcdfbde8-9474-44d7-a658-c09804d4757e, 18, Finished, Available, Finished)

In [15]:
# appending the data to the gold table
df_with_location_sig_class.write.mode('append').saveAsTable('earthquake_events_gold')

StatementMeta(, fcdfbde8-9474-44d7-a658-c09804d4757e, 19, Finished, Available, Finished)

In [ ]:
import pandas as pd
# Load data into pandas DataFrame from "/lakehouse/default/Files/2025-05-25_earthquake_data.json"
df = pd.read_json("/lakehouse/default/Files/2025-05-25_earthquake_data.json",typ="series")
display(df)


In [ ]:
import pandas as pd
# Load data into pandas DataFrame from "/lakehouse/default/Files/2025-05-25_earthquake_data.json"
df = pd.read_json("/lakehouse/default/Files/2025-05-25_earthquake_data.json",typ="series")
display(df)
